# Acknowledgements
https://www.kaggle.com/allunia/pulmonary-dicom-preprocessing - DICOM preprocessing

https://www.kaggle.com/seraphwedd18/pe-detection-with-keras-model-creation - DICOM preprocessing

https://www.kaggle.com/redwankarimsony/rsna-str-pe-gradient-sigmoid-windowing/comments - DICOM windowing

# **IMPORTING DATA**

In [31]:
import os
import pydicom
import vtk
import cv2

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

import pylab as pl
import numpy as np
import pandas as pd

from pathlib import Path
from time import time
from random import shuffle, sample, shuffle, randrange
from vtk.util import numpy_support

In [32]:
DATA_ROOT = Path("../input/rsna-str-pulmonary-embolism-detection")
TRAIN_ROOT = DATA_ROOT/"train"
TEST_ROOT = DATA_ROOT/'test'

IMAGE_RESOLUTION = (256, 256)

train_csv = pd.read_csv(DATA_ROOT/"train.csv")

In [33]:
reader = vtk.vtkDICOMImageReader()
def get_img(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

    ConstPixelSpacing = reader.GetPixelSpacing()
    
    dcm_fields = [reader.GetRescaleSlope(), reader.GetRescaleOffset()]
    
    imageData = reader.GetOutput()
    pointData = imageData.GetPointData()
    arrayData = pointData.GetArray(0)
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    ArrayDicom = cv2.resize(ArrayDicom, IMAGE_RESOLUTION)
    return ArrayDicom, dcm_fields

# **Data Processing**

In [34]:
# Preprocessing the image
# Windowing rescales the image to highlight different parts of the image

def lung_window(img, dcm_fields):
    width = 1600
    length = -600
    window_min = length - (width/2)
    window_max = length + (width/2)
    slope, intercept = dcm_fields
    #img += np.abs(np.min(img))
    img = img * slope + intercept
    img[img < window_min] = window_min
    img[img > window_max] = window_max
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    #print(np.min(img), np.max(img))
    return img

def map_to_gradient(grey_img):
    rainbow_img = np.zeros((grey_img.shape[0], grey_img.shape[1], 3))
    rainbow_img[:, :, 0] = np.clip(4 * grey_img - 2, 0, 1.0) * (grey_img > 0) * (grey_img <= 1.0)
    rainbow_img[:, :, 1] =  np.clip(4 * grey_img * (grey_img <=0.75), 0,1) + np.clip((-4*grey_img + 4) * (grey_img > 0.75), 0, 1)
    rainbow_img[:, :, 2] = np.clip(-4 * grey_img + 2, 0, 1.0) * (grey_img > 0) * (grey_img <= 1.0)
    return rainbow_img

def rainbow_window(img, dcm_fields):
    grey_img = lung_window(img, dcm_fields)
    return map_to_gradient(grey_img)

def all_channels_window(img, dcm_fields):
    grey_img = lung_window(img, dcm_fields) * 3.0
    all_chan_img = np.zeros((grey_img.shape[0], grey_img.shape[1], 3))
    all_chan_img[:, :, 2] = np.clip(grey_img, 0.0, 1.0)
    all_chan_img[:, :, 0] = np.clip(grey_img - 1.0, 0.0, 1.0)
    all_chan_img[:, :, 1] = np.clip(grey_img - 2.0, 0.0, 1.0)
    return all_chan_img

In [35]:
# Data loading functions

studies = os.listdir(TRAIN_ROOT)
#studies = studies[:len(studies)//16]

func = lambda x: int((2**15 + x)*(255/2**16))
int16_to_uint8 = np.vectorize(func)

def load_scans(dcm_path):
    # otherwise we sort by ImagePositionPatient (z-coordinate) or by SliceLocation
    slices = []
    fields = []
    for file in os.listdir(dcm_path):
        image, dcm_fields = get_img(dcm_path + "/" + file)
        #image = rainbow_window(image, dcm_fields)
        slices.append(image)
        fields.append(dcm_fields)

    return slices, fields

def filter_scanner(raw_pixelarrays):
    # in OSIC we find outside-scanner-regions with raw-values of -2000. 
    # Let's threshold between air (0) and this default (-2000) using -1000
    raw_pixelarrays[raw_pixelarrays <= -1000] = -1000
    return raw_pixelarrays

index = 125

def load_scans_from_study(study):
    scans = []
    fields = []
    series = []
    for s in os.listdir(TRAIN_ROOT/study):
        series.append(s)
        scan_set, dcm_fields = load_scans(str(TRAIN_ROOT/study/s))
        scans.append(scan_set)
        fields.append(dcm_fields)
        
    return series, scans, fields

def load_individual_scan(scan_path):
    scan, fields = get_img(scan_path)
    scan = rainbow_window(scan, fields)
    return scan

def load_batch_scans(scan_paths):
    scans = np.zeros((len(scan_paths), IMAGE_RESOLUTION[0], IMAGE_RESOLUTION[1], 3))
    for i, path in enumerate(scan_paths):
        s, f = get_img(path)
        s = rainbow_window(s, f)
        scans[i] = s
        
    return scans

In [36]:
# t0 = time()
# scans, fields = load_scans(str(TRAIN_ROOT/studies[100]/os.listdir(TRAIN_ROOT/studies[100])[0]))
# t1 = time()
# print(t1 - t0)
# s = scans[index]
# #print(s)
# pl.imshow(s, cmap=pl.cm.bone)
# pl.show()


# t0 = time()
# s = rainbow_window(scans[index], fields[index])
# t1 = time()
# print(t1 - t0)
# #print(s)
# pl.imshow(s, cmap=pl.cm.bone)
# pl.show()

# **TFRecords**

In [37]:
# TFRecord formats

UNLABELED_TFRECORD_FORMAT = {'SpecificCharacterSet': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'ImageType': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SOPClassUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SOPInstanceUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'Modality': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SliceThickness': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'KVP': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'GantryDetectorTilt': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'TableHeight': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'RotationDirection': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'XRayTubeCurrent': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'Exposure': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'ConvolutionKernel': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'PatientPosition': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'StudyInstanceUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SeriesInstanceUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SeriesNumber': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'InstanceNumber': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'ImagePositionPatient': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'ImageOrientationPatient': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'FrameOfReferenceUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SamplesPerPixel': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'PhotometricInterpretation': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'Rows': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'Columns': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'PixelSpacing': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'BitsAllocated': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'BitsStored': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'HighBit': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'PixelRepresentation': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'WindowCenter': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'WindowWidth': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'RescaleIntercept': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'RescaleSlope': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'image': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None)}

LABELED_TFRECORD_FORMAT = {'SpecificCharacterSet': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'ImageType': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SOPClassUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SOPInstanceUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'Modality': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SliceThickness': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'KVP': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'GantryDetectorTilt': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'TableHeight': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'RotationDirection': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'XRayTubeCurrent': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'Exposure': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'ConvolutionKernel': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'PatientPosition': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'StudyInstanceUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SeriesInstanceUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SeriesNumber': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'InstanceNumber': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'ImagePositionPatient': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'ImageOrientationPatient': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'FrameOfReferenceUID': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'SamplesPerPixel': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'PhotometricInterpretation': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'Rows': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'Columns': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'PixelSpacing': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'BitsAllocated': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'BitsStored': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'HighBit': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'PixelRepresentation': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'WindowCenter': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'WindowWidth': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'RescaleIntercept': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'RescaleSlope': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'negative_exam_for_pe': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'qa_motion': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'qa_contrast': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'flow_artifact': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'rv_lv_ratio_gte_1': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'rv_lv_ratio_lt_1': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'leftsided_pe': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'chronic_pe': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'true_filling_defect_not_pe': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'rightsided_pe': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'acute_and_chronic_pe': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'central_pe': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'indeterminate': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'pe_present_on_image': tf.io.FixedLenFeature(shape=(), dtype=tf.int64, default_value=None),
 'image': tf.io.FixedLenFeature(shape=(), dtype=tf.string, default_value=None)}

In [38]:
# TFRecord data loaders

def read_labeled_tfrecord(example):
    return read_tfrecord(example, LABELED_TFRECORD_FORMAT)

def read_unlabeled_tfrecord(example):
    return read_tfrecord(example, UNLABELED_TFRECORD_FORMAT)

def read_tfrecord(example, record_format):
    try:
        example = tf.io.parse_single_example(example, record_format)
    except:
        print (example)
        raise
    
    data = {k:tf.cast(example[k], record_format[k].dtype) for k in example}
        
    return data

def load_dataset(filenames, batch_size, repeat=True, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord).batch(batch_size, drop_remainder=True)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset.repeat() if repeat else dataset

In [39]:
# Preprocessing function to transform the raw numpy images
# to scaled, windowed, images.

def preprocess(data):
    image = np.array([np.frombuffer(im.numpy(), dtype=np.int16) for im in data['image']]).reshape((batch_size, 512, 512, 1))
    image = tf.image.resize(image, (IMAGE_RESOLUTION[0], IMAGE_RESOLUTION[1])).numpy()
    image = np.squeeze(image, -1)
    
    out_images = np.zeros((batch_size, IMAGE_RESOLUTION[0], IMAGE_RESOLUTION[1], 3))
    
    for i, im in enumerate(image):
        dcm_fields = (int(float(data['RescaleSlope'].numpy()[i].decode())), int(float(data['RescaleIntercept'].numpy()[i].decode())))
        out_images[i] = rainbow_window(image[i], dcm_fields)

    label = data['pe_present_on_image'].numpy()
    label = tf.reshape(label, (batch_size, 1))
    
    return out_images, label

# **Splitting Data**

In [40]:
MAX_IMAGES = 500000

image_labels = train_csv[['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID', 'pe_present_on_image']]

exam_labels = train_csv[['StudyInstanceUID', 'negative_exam_for_pe', 'rv_lv_ratio_gte_1',
                         'rv_lv_ratio_lt_1', 'leftsided_pe', 'chronic_pe', 'rightsided_pe',
                         'acute_and_chronic_pe', 'central_pe', 'indeterminate']].drop_duplicates('StudyInstanceUID')

In [41]:
negative_indices = list(image_labels.loc[image_labels['pe_present_on_image'] == 0].axes[0])
positive_indices = list(image_labels.loc[image_labels['pe_present_on_image'] == 1].axes[0])

all_indices = negative_indices + positive_indices

# **Model**

In [42]:
batch_size = 24
epochs = 2
learning_rate = 1e-6
image_opt = keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True)

In [43]:
# Model definitions

def get_model():
    image_inp = keras.Input(shape=(IMAGE_RESOLUTION[0], IMAGE_RESOLUTION[1], 3))
    image_model = keras.applications.EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')(image_inp)
    image_model = layers.Dense(1, activation='sigmoid')(image_model)
    image_model = keras.Model(inputs=image_inp, outputs=image_model)

    image_model.summary()
    
    return image_model

In [44]:
# Loss weights to apply to the calculated loss
# Ratio of all the training samples to the positive training samples
loss_weights = np.full((batch_size, 1), len(all_indices)/len(positive_indices))

# Loss function with the option of weighting classes
# Uses the defined loss_weights above
def loss_function(labels, logits, weighted=True):
    loss = keras.losses.binary_crossentropy(labels, logits, from_logits=True, label_smoothing=0.05)
    
    if weighted:
        weights = tf.cast(tf.math.greater(labels, 0), tf.float32)*loss_weights
        weights += tf.cast(tf.math.equal(weights, 0), tf.float32)
        weights = tf.reshape(weights, (weights.shape[0],))
        
        loss = tf.math.multiply(loss, weights)

    loss = tf.math.reduce_mean(loss)
    
    return loss

#@tf.function
def image_train_step(image_model, image, labels):
    with tf.GradientTape() as tape:
        logits = image_model(image)
        loss = loss_function(labels, logits, weighted=True)
        
    gradients = tape.gradient(loss, image_model.trainable_variables)
    image_opt.apply_gradients(zip(gradients, image_model.trainable_variables))
    
    return loss, logits

#@tf.function
def image_eval_step(image_model, image, labels):
    logits = image_model(image)
    loss = loss_function(labels, logits, weighted=True)
    
    return loss, logits

train_root = "../input/rsna-pe-tfrecords-v2/train/"
all_tfrecords = [train_root+record for record in os.listdir(train_root)]

# Cross Validation

In [45]:
# doing k-fold cross validation from the full training dataset to preserve the skewed ratio
k = 5
fold_size = 50000
images_per_record = 559

# generating folds
def generate_folds(k, fold_size, tfrecords):
    assert fold_size < len(tfrecords) * images_per_record / k
    
    # fold_size can be set to -1 to use all TFRecords
    if fold_size == -1:
        records_per_fold = len(tfrecords) // k
    else:
        records_per_fold = fold_size // images_per_record

    folds = list()
    for i in range(k):
        shuffle(tfrecords)

        fold = tfrecords[:records_per_fold]
        folds.append(fold)

        tfrecords = tfrecords[records_per_fold:]
    
    return folds
    
# running CV
def cross_validation_training(folds):
    loss_met = keras.metrics.Mean()
    auc_pr_met = keras.metrics.BinaryAccuracy()

    count = 0

    validation_history = []
    for i in range(len(folds)):
        keras.backend.clear_session()
        
        image_model = get_model()
        
        prev = folds[:i]
        post = folds[i+1:] if i < len(folds)-1 else None
        train_folds = prev + post if post is not None else prev
        train_dataset = []
        # there's probably a better way of doing this
        for f in train_folds:
            for record in f:
                train_dataset.append(record)

        train_dataset = load_dataset(train_dataset, batch_size, repeat=False)

        validation_dataset = folds[i]
        validation_dataset = load_dataset(validation_dataset, batch_size, repeat=False)

        for e in range(epochs):
            for n, data in enumerate(train_dataset):
                # preprocessing before feeding to model
                image, label = preprocess(data)

                # feed-forward and backprop
                loss, logits = image_train_step(image_model, image, label)

                loss_met(loss)
                auc_pr_met(label, logits)

                print(f"Epoch {e} of {epochs}, training data point {n+1} average loss, accuracy: {loss_met.result()}, {auc_pr_met.result()}    \r", end='')

            print(f"Epoch {e} of {epochs}, fold {i+1} training average loss, accuracy: {loss_met.result()}, {auc_pr_met.result()}")
            
        print(f"Fold {i+1} training average loss, accuracy: {loss_met.result()}, {auc_pr_met.result()}")
        print("------------------")

        loss_met.reset_states()
        auc_pr_met.reset_states()
        for n, data in enumerate(validation_dataset):
            # preprocessing before feeding to model
            image, label = preprocess(data)

            # feed-forward and backprop
            loss, logits = image_eval_step(image_model, image, label)

            loss_met(loss)
            auc_pr_met(label, logits)

            print(f"Validation data point {n+1} average loss, accuracy: {loss_met.result()}, {auc_pr_met.result()}    \r", end='')
            
            
        validation_history.append(loss_met.result())
        print(f"Fold {i+1} validation average loss, accuracy: {loss_met.result()}, {auc_pr_met.result()}               ")
        print("------------------")
        
        image_model.save_weights(f"image_weights_{i+1}.h5")
        
        del image_model
        
    return validation_history

folds = generate_folds(k, fold_size, all_tfrecords)
validation_history = cross_validation_training(folds)

for i, validation in enumerate(validation_history):
    print(f"Fold {i+1} validation: {validation}")

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________
Epoch 0 of 2, fold 1 training average loss, AUC: 1.6315443515777588, 0.63844084739685063968506     
Epoch 1 of 2, fold 1 training average loss, AUC: 1.6289920806884766, 0.77822577953338625333862    
Fold 1 training average loss, AUC: 1.6289920806884766, 0.7782257795333862
------------------
Fold 1 validation average loss, AUC: 1.4268226623535156, 0.942028999328613332